In [4]:
import numpy as np
import matplotlib.pyplot as plt
num_elements = 300
elements = np.random.normal(0.5, 0.1, (num_elements, 3)) # three columns for index, power, and channel gain
noise_power = 0.01 # noise power density
population_size = 100
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1
user_x = 100 # user x coordinate
user_y = 0 # user y coordinate
ris_x = 50 # UAV-RIS x coordinate
ris_y = 50 # UAV-RIS y coordinate
bs_x = 0 # base station x coordinate
bs_y = 0 # base station y coordinate
initial_snr_without_ris = elements[:, 1] / (noise_power * np.sqrt((elements[:, 0] - user_x) ** 2 + (elements[:, 1] - user_y) ** 2)) # SNR without RIS
def calculate_snr_power(energy_indices, signal_indices):
    energy_power = np.sum(elements[energy_indices, 1]) # total power of energy elements
    signal_power = np.sum(elements[signal_indices, 1]) # total power of signal elements
    energy_channel_gain = np.sum(elements[energy_indices, 2]) # total channel gain of energy elements
    signal_channel_gain = np.sum(elements[signal_indices, 2]) # total channel gain of signal elements
    energy_distance_from_user = np.sum(np.sqrt((elements[energy_indices, 0] - user_x) ** 2 + (elements[energy_indices, 1] - user_y) ** 2)) # total distance of energy elements from user
    signal_distance_from_user = np.sum(np.sqrt((elements[signal_indices, 0] - user_x) ** 2 + (elements[signal_indices, 1] - user_y) ** 2)) # total distance of signal elements from user
    energy_distance_from_ris = np.sum(np.sqrt((elements[energy_indices, 0] - ris_x) ** 2 + (elements[energy_indices, 1] - ris_y) ** 2)) # total distance of energy elements from UAV-RIS
    signal_distance_from_ris = np.sum(np.sqrt((elements[signal_indices, 0] - ris_x) ** 2 + (elements[signal_indices, 1] - ris_y) **2)) # total distance of signal elements from UAV-RIS
    energy_distance_from_bs = np.sum(np.sqrt((elements[energy_indices, 0] - bs_x) ** 2 + (elements[energy_indices, 1] - bs_y) ** 2)) # total distance of energy elements from base station
    signal_distance_from_bs = np.sum(np.sqrt((elements[signal_indices, 0] - bs_x) ** 2 + (elements[signal_indices, 1] - bs_y) ** 2)) # total distance of signal elements from base station
    theta = np.random.uniform(0, 2 * np.pi, num_elements) # random phase shifts of RIS elements
    snr = (signal_power * signal_channel_gain * np.cos(theta[signal_indices]) / (signal_distance_from_user ** 2 + signal_distance_from_ris ** 2 + signal_distance_from_bs ** 2 + 1e-10)) / (energy_power * energy_channel_gain * np.cos(theta[energy_indices]) / (energy_distance_from_user ** 2 + energy_distance_from_ris ** 2 + energy_distance_from_bs ** 2 + 1e-10) + noise_power)
    snr *= np.random.rayleigh() # fading channel gain
    power = energy_power + signal_power
    return snr, power

def evaluate_fitness(energy_indices, signal_indices):
    snr, power = calculate_snr_power(energy_indices, signal_indices)
    return snr * power # fitness function

def genetic_algorithm():
    population = [np.random.choice(range(num_elements), num_elements // 2, replace=False) for _ in range(population_size)] # initial population
    for _ in range(num_generations): # main loop
        new_population = []
        for parent1, parent2 in zip(population[::2], population[1::2]): # pairwise crossover
            if np.random.rand() < crossover_rate: # crossover probability
                crossover_point = np.random.randint(1, num_elements // 2) # random crossover point
                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:])) # first child
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:])) # second child
            else: # no crossover
                child1 = parent1 # first child is same as first parent
                child2 = parent2 # second child is same as second parent
            for child in [child1, child2]: # mutation
                if np.random.rand() < mutation_rate: # mutation probability
                    mutation_index = np.random.randint(num_elements // 2) # random mutation index
                    child[mutation_index] = np.random.randint(num_elements) # random mutation value
            new_population.extend([child1, child2]) # add children to new population
        population = new_population # update population
    best_individual = min(population, key=lambda ind: evaluate_fitness(ind, list(set(range(num_elements)) - set(ind)))) # find best individual
    best_energy_indices = best_individual # best energy indices
    best_signal_indices = list(set(range(num_elements)) - set(best_individual)) # best signal indices
    best_snr, best_power = calculate_snr_power(best_energy_indices, best_signal_indices) # best SNR and power
    return best_energy_indices, best_signal_indices, best_snr, best_power

def smarter_optimization(energy_indices, signal_indices):
    initial_snr, initial_power = calculate_snr_power(energy_indices, signal_indices) # initial SNR and power
    best_energy_indices = energy_indices.copy() # best energy indices
    best_signal_indices = signal_indices.copy() # best signal indices
    best_combined_improvement = 0 # best combined improvement metric
    for _ in range(100): # main loop 
        improved = False # flag for improvement
        for energy_index in energy_indices: # loop over energy indices
            for signal_index in signal_indices: # loop over signal indices
                new_energy_indices = [idx for idx in energy_indices if idx != energy_index] # new energy indices by swapping one element
                new_signal_indices = [idx for idx in signal_indices if idx != signal_index] # new signal indices by swapping one element
                new_energy_indices.append(signal_index) # add swapped element to new energy indices
                new_signal_indices.append(energy_index) # add swapped element to new signal indices
                new_snr, new_power = calculate_snr_power(new_energy_indices, new_signal_indices) # new SNR and power
                snr_improvement = new_snr - initial_snr # SNR improvement
                power_reduction = new_power - initial_power # power reduction
                combined_improvement_metric = snr_improvement - 0.5 * power_reduction # combined improvement metric
                if combined_improvement_metric > best_combined_improvement: # if improvement is better than best
                    best_combined_improvement = combined_improvement_metric # update best improvement
                    best_energy_indices = new_energy_indices # update best energy indices
                    best_signal_indices = new_signal_indices # update best signal indices
                    improved = True # set improvement flag to true
        if not improved: # if no improvement
            break # stop the loop
    return best_energy_indices, best_signal_indices, new_snr, new_power # return best energy indices, best signal indices, new SNR and new power

best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm() # run genetic algorithm
best_energy_indices, best_signal_indices, improved_snr, improved_power = smarter_optimization(best_energy_indices, best_signal_indices) # run smarter optimization
initial_snr, initial_power = calculate_snr_power(range(num_elements // 2), range(num_elements // 2, num_elements)) # initial SNR and power without RIS
improved_snr_with_ris = improved_snr / initial_snr_without_ris # improved SNR with RIS

print("Initial SNR without RIS:", initial_snr_without_ris)
print("Initial SNR with RIS:", initial_snr)
print("Initial power with RIS:", initial_power)
print("Improved SNR with RIS:", improved_snr)
print("Improved power with RIS:", improved_power)
print("Improved SNR with RIS over initial SNR without RIS:", improved_snr_with_ris)



ValueError: operands could not be broadcast together with shapes (172,) (150,) 

In [10]:
# import numpy library
import numpy as np

# define number of elements
num_elements = 100

# define number of generations
num_generations = 50

# define number of individuals in population
num_individuals = 20

# define crossover probability
crossover_prob = 0.8

# define mutation probability
mutation_prob = 0.2

# define initial SNR without RIS
initial_snr_without_ris = 10

# define initial power without RIS
initial_power_without_ris = 1

# define noise power
noise_power = 0.01

# define channel gain matrix from RIS to receiver
H_ris_rx = np.random.randn(num_elements, num_elements)

# define channel gain matrix from transmitter to RIS
H_tx_ris = np.random.randn(num_elements, num_elements)

# define channel gain matrix from transmitter to receiver
H_tx_rx = np.random.randn(num_elements, num_elements)

# define function to calculate SNR and power with given energy indices and signal indices
def calculate_snr_power(energy_indices, signal_indices):
    # create energy vector with ones at energy indices and zeros elsewhere
    energy_vector = np.zeros(num_elements)
    energy_vector[energy_indices] = 1
    
    # create signal vector with ones at signal indices and zeros elsewhere
    signal_vector = np.zeros(num_elements)
    signal_vector[signal_indices] = 1
    
    # calculate effective channel gain from transmitter to receiver with RIS
    H_eff = H_tx_rx + np.dot(H_tx_ris, np.diag(energy_vector)).dot(H_ris_rx)
    
    # calculate signal power at receiver with RIS
    signal_power = np.sum(np.square(np.abs(np.dot(H_eff, signal_vector))))
    
    # calculate SNR at receiver with RIS
    snr = (signal_power * signal_channel_gain) / noise_power
    
    # calculate power consumed by RIS elements
    power = np.sum(energy_vector)
    
    # return SNR and power
    return snr, power

# define function to evaluate fitness of an individual (energy indices)
def evaluate_fitness(energy_indices, signal_indices):
    # calculate SNR and power with given energy indices and signal indices
    snr, power = calculate_snr_power(energy_indices, signal_indices)
    
    # calculate SNR improvement over initial SNR without RIS
    snr_improvement = snr / initial_snr_without_ris
    
    # calculate power reduction over initial power without RIS
    power_reduction = initial_power_without_ris - power
    
    # calculate combined fitness metric as SNR improvement minus half of power reduction
    fitness = snr_improvement - 0.5 * power_reduction
    
    # return fitness
    return fitness

# define function to initialize population randomly
def initialize_population():
    # create empty list for population
    population = []
    
    # loop for number of individuals in population
    for i in range(num_individuals):
        # randomly select half of the elements as energy indices
        energy_indices = np.random.choice(range(num_elements), size=num_elements // 2, replace=False)
        
        # add energy indices to population as an individual
        population.append(energy_indices)
        
    # return population
    return population

# define function to perform crossover between two parents
def crossover(parent1, parent2):
    # randomly select a crossover point between 1 and number of elements - 1
    crossover_point = np.random.randint(1, num_elements - 1)
    
    # create child1 by taking first part of parent1 and second part of parent2
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    
    # create child2 by taking first part of parent2 and second part of parent1
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    # return children
    return child1, child2

# define function to perform mutation on an individual
def mutation(individual):
    # randomly select two elements to swap
    swap_indices = np.random.choice(range(num_elements), size=2, replace=False)
    
    # swap the elements in the individual array
    individual[swap_indices[0]], individual[swap_indices[1]] = individual[swap_indices[1]], individual[swap_indices[0]]
    
    # return mutated individual
    return individual

# define function to perform genetic algorithm
def genetic_algorithm():
    # initialize population randomly
    population = initialize_population()
    
    # loop for number of generations
    for g in range(num_generations):
        # create empty list for new population
        new_population = []
        
        # loop for half of the number of individuals in population
        for i in range(num_individuals // 2):
            # randomly select two parents from population
            parent1, parent2 = np.random.choice(population, size=2, replace=False)
            
            # generate a random number between 0 and 1
            r = np.random.rand()
            
            # if random number is less than crossover probability, perform crossover
            if r < crossover_prob:
                # perform crossover between parents and get children
                child1, child2 = crossover(parent1, parent2)
            else:
                # otherwise, copy parents as children
                child1, child2 = parent1.copy(), parent2.copy()
            
            # generate another random number between 0 and 1
            r = np.random.rand()
            
            # if random number is less than mutation probability, perform mutation on child1
            if r < mutation_prob:
                # perform mutation on child1
                child1 = mutation(child1)
            
            # generate another random number between 0 and 1
            r = np.random.rand()
            
            # if random number is less than mutation probability, perform mutation on child2
            if r < mutation_prob:
                # perform mutation on child2
                child2 = mutation(child2)
            
            # add children to new population
            new_population.extend([child1, child2])
        
        # update population with new population
        population = new_population
    
    # find best individual in population by minimizing fitness function
    best_individual = min(population, key=lambda ind: evaluate_fitness(ind, list(set(range(num_elements)) - set(ind))))
    
    # get best energy indices from best individual
    best_energy_indices = best_individual
    
    # get best signal indices from complement of best individual
    best_signal_indices = list(set(range(num_elements)) - set(best_individual))
    
    # calculate SNR and power with best energy indices and best signal indices
    best_snr, best_power = calculate_snr_power(best_energy_indices, best_signal_indices)
    
    # return best energy indices, best signal indices, best SNR and best power
    return best_energy_indices, best_signal_indices, best_snr, best_power

# define function to perform smarter optimization after genetic algorithm
def smarter_optimization(energy_indices, signal_indices):
    # calculate initial SNR and power with given energy indices and signal indices
    initial_snr, initial_power = calculate_snr_power(energy_indices, signal_indices)
    
    # create new energy indices and new signal indices by copying given ones
    new_energy_indices = energy_indices.copy()
    new_signal_indices = signal_indices.copy()
    
    # loop for number of elements times 10 (arbitrary number of iterations)
    for i in range(num_elements * 10):
        # randomly select an element from energy indices and an element from signal indices to swap
        swap_energy_index = np.random.choice(new_energy_indices)
        swap_signal_index = np.random.choice(new_signal_indices)
        
        # swap the elements in the new energy indices and new signal indices arrays
        new_energy_indices[new_energy_indices.index(swap_energy_index)] = swap_signal_index
        new_signal_indices[new_signal_indices.index(swap_signal_index)] = swap_energy_index
        
        # calculate new SNR and power with swapped energy indices and signal indices
        new_snr, new_power = calculate_snr_power(new_energy_indices, new_signal_indices)
        
        # if new SNR is greater than or equal to initial SNR and new power is less than or equal to initial power, accept the swap
        if (new_snr >= initial_snr) and (new_power <= initial_power):
            # update initial SNR and power with new SNR and power
            initial_snr = new_snr
            initial_power = new_power
        
        else:
            # otherwise, undo the swap by swapping back the elements in the new energy indices and new signal indices arrays
            new_energy_indices[new_energy_indices.index(swap_signal_index)] = swap_energy_index
            new_signal_indices[new_signal_indices.index(swap_energy_index)] = swap_signal_index
        
        # if initial power is zero, break the loop (cannot reduce power further)
        if initial_power == 0:
            break
    
    # return new energy indices, new signal indices, new SNR and new power after optimization
    return new_energy_indices, new_signal_indices, new_snr, new_power

# run genetic algorithm and get best
# run genetic algorithm and get best energy indices, best signal indices, best SNR and best power
best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# print best energy indices, best signal indices, best SNR and best power
print("Best energy indices:", best_energy_indices)
print("Best signal indices:", best_signal_indices)
print("Best SNR:", best_snr)
print("Best power:", best_power)

# run smarter optimization after genetic algorithm and get optimized energy indices, optimized signal indices, optimized SNR and optimized power
optimized_energy_indices, optimized_signal_indices, optimized_snr, optimized_power = smarter_optimization(best_energy_indices, best_signal_indices)

# print optimized energy indices, optimized signal indices, optimized SNR and optimized power
print("Optimized energy indices:", optimized_energy_indices)
print("Optimized signal indices:", optimized_signal_indices)
print("Optimized SNR:", optimized_snr)
print("Optimized power:", optimized_power)


ValueError: a must be 1-dimensional